# **Stable Diffusion**
- deepface to crop only to head
- Duplicate number of heads
- resize input images to 1024 1024
- Add tensorboard? Track live traning

## **Set up**

In [11]:
# !git clone https://github.com/bmaltais/kohya_ss.git /home/datascience/kohya_ss
# !mkdir -p /home/datascience/stable-diffusion-xl-base-1.0 && wget -O /home/datascience/stable-diffusion-xl-base-1.0/sd_xl_base_1.0.safetensors https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors

# !bash /home/datascience/kohya_ss/setup.sh
# !source /home/datascience/kohya_ss/venv/bin/activate
#!pip install deepspeed accelerate config default safetensors transformers diffusers einops toml torchvision opencv-python-headless voluptuous xformers deepface open_clip_torch
# !pip install mediapipe

# !mkdir -p /home/datascience/sks/img/1_sks\ person
# !mkdir -p /home/datascience/sks/reg
# !mkdir -p /home/datascience/sks/model
# !mkdir -p /home/datascience/sks/log

## **Train Stable Diffusion - SD XL**

In [ ]:
#remove output if so
!rm -r /home/datascience/sks/output

!bash /home/datascience/kohya_ss/setup.sh
!source /home/datascience/kohya_ss/venv/bin/activate

!accelerate launch  /home/datascience/kohya_ss/sdxl_train_network.py \
  --enable_bucket \
  --min_bucket_reso=256 \
  --max_bucket_reso=2048 \
  --pretrained_model_name_or_path="/home/datascience/stable-diffusion-xl-base-1.0/sd_xl_base_1.0.safetensors" \
  --train_data_dir="/home/datascience/sks/img" \
  --resolution="1024,1024" \
  --output_dir="/home/datascience/sks/output" \
  --network_alpha="1" \
  --save_model_as=safetensors \
  --network_module=networks.lora \
  --text_encoder_lr=0.0004 \
  --unet_lr=0.0004 \
  --network_dim=256 \
  --output_name="sks" \
  --lr_scheduler_num_cycles="10" \
  --no_half_vae --learning_rate="0.0004" \
  --lr_scheduler="cosine" \
  --train_batch_size="1" \
  --max_train_steps="800" \
  --mixed_precision="fp16" \
  --save_precision="fp16" \
  --optimizer_type="adafactor" \
  --cache_latents \
  --cache_latents_to_disk \
  --gradient_checkpointing \
  --optimizer_args scale_parameter=False relative_step=False warmup_init=False \
  --max_data_loader_n_workers="0" \
  --bucket_reso_steps=64 \
  --xformers \
  --bucket_no_upscale

---

## **SD XL Inference**

In [ ]:
#some documentation: https://github.com/huggingface/diffusers/pull/4287 and https://huggingface.co/docs/diffusers/main/en/training/lora#supporting-a1111-themed-lora-checkpoints-from-diffusers

In [ ]:
!python /home/datascience/kohya_ss/sdxl_minimal_inference.py --ckpt_path /home/datascience/stable-diffusion-xl-base-1.0/sd_xl_base_1.0.safetensors --output_dir=. --lora_weights /home/datascience/sks/output/sks.safetensors --prompt "ultra realistic illustration, ((sks man)) in a red bull racing formula 1 suit intricate, elegant, no helmet, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha"

---

# **OCI Data Science - Jobs**

In [ ]:
#copy folder/files into job artifacts
# !cp -R /home/datascience/kohya_ss /home/datascience/job_artifact

## **Publish Conda**

In [5]:
#1.st define bucket and namespace where to store  (conda_environment_yolov5 = bucket name, frqap2zhtzbe = namespace)
!odsc conda init -b West_BP -n frqap2zhtzbe -a resource_principal

In [4]:
!odsc conda publish -s pytorch20_p39_gpu_v1 --force

# **Create a Job**

In [4]:
from ads.common.oci_logging import OCILogGroup, OCILog
from ads.jobs import Job, DataScienceJob, PythonRuntime
from datetime import datetime, timedelta
from ads import set_auth

#authentication
from ads import set_auth
set_auth(auth='resource_principal')

In [5]:
## the below script creates an OCI Data Science - Job
job = (
    Job(name="stable_dff_v1")
    .with_infrastructure(
        DataScienceJob()
        # Configure logging for getting the job run outputs.
        .with_log_group_id("ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdyajxalcuggjaug57r3ugare7olsk44ts2shyv7azqbxf4q")
        .with_shape_name("VM.GPU.A10.1")
        #.with_shape_config_details(memory_in_gbs=16, ocpus=5)
        .with_block_storage_size(1000)
    )
    .with_runtime(
        PythonRuntime()
        .with_custom_conda("oci://West_BP@frqap2zhtzbe/conda_environments/gpu/PyTorch 2.0 for GPU on Python 3.9/1.0/pytorch20_p39_gpu_v1")
        .with_source("/home/datascience/job_artifact/")  
        .with_entrypoint("stable_main.py")  
        .with_working_dir("job_artifact")
        .with_environment_variable(full_input_folder="oci://West_BP@frqap2zhtzbe/stable_diffusion/input_folder/")
        .with_output("./inference_output", "oci://West_BP@frqap2zhtzbe/stable_diffusion/output_folder/")  #copy output in inference_output to output_folder bucket
    )
)

job.create()


kind: job
spec:
  id: ocid1.datasciencejob.oc1.eu-frankfurt-1.amaaaaaangencdyamddgmkbvwn5aazhufilhbp2v2mp6o6mn5wsg35tws3xa
  infrastructure:
    kind: infrastructure
    spec:
      blockStorageSize: 1000
      compartmentId: ocid1.compartment.oc1..aaaaaaaae3n6r6hrjipbap2hojicrsvkzatrtlwvsyrpyjd7wjnw4za3m75q
      displayName: stable_dff_v1
      jobInfrastructureType: ME_STANDALONE
      jobType: DEFAULT
      logGroupId: ocid1.loggroup.oc1.eu-frankfurt-1.amaaaaaangencdyajxalcuggjaug57r3ugare7olsk44ts2shyv7azqbxf4q
      projectId: ocid1.datascienceproject.oc1.eu-frankfurt-1.amaaaaaangencdyaik5ssdqk4as2bhldxprh7vnqpk7yycsm7vymd344cgua
      shapeName: VM.GPU.A10.1
    type: dataScienceJob
  name: stable_dff_v1
  runtime:
    kind: runtime
    spec:
      conda:
        type: published
        uri: oci://West_BP@frqap2zhtzbe/conda_environments/gpu/PyTorch 2.0 for GPU
          on Python 3.9/1.0/pytorch20_p39_gpu_v1
      entrypoint: stable_main.py
      env:
      - name: full_input_f

In [6]:
job_run_env = job.run(
    name="Job Run - Passing dynamic values",
    env_var={'full_input_folder': 'oci://West_BP@frqap2zhtzbe/stable_diffusion/input_folder/'}
)

job_run_watch = job_run_env.watch()

Job OCID: ocid1.datasciencejob.oc1.eu-frankfurt-1.amaaaaaangencdyamddgmkbvwn5aazhufilhbp2v2mp6o6mn5wsg35tws3xa
Job Run OCID: ocid1.datasciencejobrun.oc1.eu-frankfurt-1.amaaaaaangencdyafo7ajs5hss45nu5gbky6s2zvigo5vmqvrktj4t6a6tbq
2023-08-08 18:08:00 - Job Run ACCEPTED
2023-08-08 18:08:13 - Job Run ACCEPTED, Infrastructure provisioning.
2023-08-08 18:09:13 - Job Run ACCEPTED, Infrastructure provisioned.
2023-08-08 18:09:48 - Job Run ACCEPTED, Job run bootstrap starting.
2023-08-08 18:13:35 - Job Run ACCEPTED, Job run bootstrap complete. Artifact execution starting.
2023-08-08 18:13:45 - Job Run IN_PROGRESS, Job run artifact execution in progress.
2023-08-08 18:13:35 - Full input folder used is oci://West_BP@frqap2zhtzbe/stable_diffusion/input_folder/
2023-08-08 18:13:35 - --2023-08-08 18:13:35--  https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors
2023-08-08 18:13:35 - Resolving huggingface.co (huggingface.co)... 143.204.215.22, 143.204

In [32]:
import argparse
import cv2
import numpy as np
import secrets
import os


from job_artifact.sub_packages.face_cropper import FaceCropper
#from sub_packages.face_cropper import FaceCropper

#list of local images
list_of_local_images = os.listdir("testimages")
print(list_of_local_images)
count = 1

#crop each image to face only and resize image
for local_images in list_of_local_images:
    
    if local_images.endswith(".jpg"):

        #crop image to face only
        face_cropper = FaceCropper()
        image_bgr = cv2.imread(("./testimages/" + local_images))
        faces_rgb = face_cropper.get_faces(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
        img_name = "./testimages/testout/img_"+str(count)

        if not faces_rgb:
            print("No faces detected")
        else:
            #resize the image
            dim = (1024, 1024)
            resized_image = cv2.resize(faces_rgb, dim, interpolation = cv2.INTER_AREA)

            print('Resized Dimensions of input image for training : ',resized_image.shape)

            #save image
            cv2.imwrite(img_name, resized_image)
            count += 1


ERROR - Exception
Traceback (most recent call last):
  File "/home/datascience/conda/pytorch20_p39_gpu_v1/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_20977/3105197982.py", line 24, in <cell line: 17>
    faces_rgb = face_cropper.get_faces(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
  File "/home/datascience/job_artifact/sub_packages/face_cropper.py", line 576, in get_faces
    inflated_face_image, face_landmarks = _get_roll_corrected_image_and_landmarks(inflated_face_image, face_landmarks)
  File "/home/datascience/job_artifact/sub_packages/face_cropper.py", line 440, in _get_roll_corrected_image_and_landmarks
    eyes_midpoint = _get_eyes_midpoint(left_eye_centre, right_eye_centre, face_image.shape)
  File "/home/datascience/job_artifact/sub_packages/face_cropper.py", line 377, in _get_eyes_midpoint
    (left_eye_centre[1] + right_eye_centre[1]) * image_size[0] / 2]))

['img_1.jpg', 'img_5.jpg', 'img_2 - Copy.jpg', 'testout', 'img_5 - Copy.jpg', 'img_2.jpg', 'img_3.jpg', 'img_3 - Copy.jpg', '.ipynb_checkpoints', 'img_1 - Copy.jpg']


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

## **Gradio**